In [1]:
num_clients = 10

In [2]:
from omegaconf import OmegaConf

server_config_file = "../../examples/resources/configs/cifar10/server_fedavg.yaml"
server_config = OmegaConf.load(server_config_file)
print(OmegaConf.to_yaml(server_config))

client_configs:
  train_configs:
    trainer: VanillaTrainer
    dp_mechanism: opacus
    dp_config:
      noise_multiplier: 1.0
      max_grad_norm: 1.0
    mode: step
    num_local_steps: 100
    optim: Adam
    optim_args:
      lr: 0.01
    loss_fn_path: ./resources/loss/celoss.py
    loss_fn_name: CELoss
    do_validation: true
    do_pre_validation: true
    metric_path: ./resources/metric/acc.py
    metric_name: accuracy
    use_dp: true
    epsilon: 1
    clip_grad: false
    clip_value: 1
    clip_norm: 1
    train_batch_size: 64
    val_batch_size: 64
    train_data_shuffle: true
    val_data_shuffle: false
  model_configs:
    model_path: ./resources/model/resnet.py
    model_name: ResNet18
  comm_configs:
    compressor_configs:
      enable_compression: false
      lossy_compressor: SZ2Compressor
      lossless_compressor: blosc
      error_bounding_mode: REL
      error_bound: 0.001
      param_cutoff: 1024
server_configs:
  num_clients: 2
  scheduler: SyncScheduler
  sch

In [3]:
server_config.client_configs.train_configs.loss_fn_path = (
    "../../examples/resources/loss/celoss.py"
)
server_config.client_configs.train_configs.metric_path = (
    "../../examples/resources/metric/acc.py"
)
server_config.client_configs.model_configs.model_path = ""

if server_config.client_configs.train_configs["use_dp"] and (
    server_config.client_configs.train_configs["dp_mechanism"] == "opacus"
):
    server_config.client_configs.model_configs.model_path = (
        "../../examples/resources/model/resnet_opacus.py"
    )
else:
    server_config.client_configs.model_configs.model_path = (
        "../../examples/resources/model/resnet.py"
    )

server_config.server_configs.num_global_epochs = 3
server_config.server_configs.num_clients = num_clients

In [4]:
client_config_file = "../../examples/resources/configs/cifar10/client_1.yaml"
client_config = OmegaConf.load(client_config_file)
print(OmegaConf.to_yaml(client_config))

client_id: Client1
train_configs:
  device: cuda
  logging_output_dirname: ./output
  logging_output_filename: result
data_configs:
  dataset_path: ./resources/dataset/cifar10_dataset.py
  dataset_name: get_cifar10
  dataset_kwargs:
    num_clients: 2
    client_id: 0
    partition_strategy: class_noniid
    visualization: true
    output_dirname: ./output
    output_filename: visualization.pdf
comm_configs:
  grpc_configs:
    server_uri: localhost:50051
    max_message_size: 1048576
    use_ssl: false



In [5]:
import copy

client_configs = [copy.deepcopy(client_config) for _ in range(num_clients)]
for i in range(num_clients):
    client_configs[i].client_id = f"Client{i + 1}"
    client_configs[
        i
    ].data_configs.dataset_path = "../../examples/resources/dataset/cifar10_dataset.py"
    client_configs[i].data_configs.dataset_kwargs.num_clients = num_clients
    client_configs[i].data_configs.dataset_kwargs.client_id = i
    client_configs[i].data_configs.dataset_kwargs.visualization = (
        True if i == 0 else False
    )

In [6]:
from appfl.agent import ServerAgent, ClientAgent

server_agent = ServerAgent(server_agent_config=server_config)
client_agents = [
    ClientAgent(client_agent_config=client_configs[i]) for i in range(num_clients)
]

appfl: ✅[2025-09-16 16:27:50,986 server]: Logging to ./output/result_Server_2025-09-16-16-27-50.txt
09/16/2025 16:27:50:INFO:Logging to ./output/result_Server_2025-09-16-16-27-50.txt
appfl: ✅[2025-09-16 16:27:51,023 Client1]: Logging to ./output/result_Client1_2025-09-16-16-27-51.txt
09/16/2025 16:27:51:INFO:Logging to ./output/result_Client1_2025-09-16-16-27-51.txt
100%|██████████| 170M/170M [00:48<00:00, 3.51MB/s] 
appfl: ✅[2025-09-16 16:28:55,826 Client2]: Logging to ./output/result_Client2_2025-09-16-16-28-55.txt
09/16/2025 16:28:55:INFO:Logging to ./output/result_Client2_2025-09-16-16-28-55.txt
appfl: ✅[2025-09-16 16:29:11,133 Client3]: Logging to ./output/result_Client3_2025-09-16-16-29-11.txt
09/16/2025 16:29:11:INFO:Logging to ./output/result_Client3_2025-09-16-16-29-11.txt
appfl: ✅[2025-09-16 16:29:26,515 Client4]: Logging to ./output/result_Client4_2025-09-16-16-29-26.txt
09/16/2025 16:29:26:INFO:Logging to ./output/result_Client4_2025-09-16-16-29-26.txt
appfl: ✅[2025-09-16 1

In [7]:
# Get additional client configurations from the server
client_config_from_server = server_agent.get_client_configs()
for client_agent in client_agents:
    client_agent.load_config(client_config_from_server)

In [8]:
# Load initial global model from the server
init_global_model = server_agent.get_parameters(serial_run=True)
for client_agent in client_agents:
    client_agent.load_parameters(init_global_model)

In [9]:
# [Optional] Set number of local data to the server
for i in range(num_clients):
    sample_size = client_agents[i].get_sample_size()
    server_agent.set_sample_size(
        client_id=client_agents[i].get_id(), sample_size=sample_size
    )

In [10]:
while not server_agent.training_finished():
    new_global_models = []
    for client_agent in client_agents:
        # Client local training
        client_agent.train()
        local_model = client_agent.get_parameters()
        if isinstance(local_model, tuple):
            local_model, metadata = local_model[0], local_model[1]
        else:
            metadata = {}
        # "Send" local model to server and get a Future object for the new global model
        # The Future object will be resolved when the server receives local models from all clients
        new_global_model_future = server_agent.global_update(
            client_id=client_agent.get_id(),
            local_model=local_model,
            blocking=False,
            **metadata,
        )
        new_global_models.append(new_global_model_future)
    # Load the new global model from the server
    for client_agent, new_global_model_future in zip(client_agents, new_global_models):
        client_agent.load_parameters(new_global_model_future.result())

AssertionError: Torch not compiled with CUDA enabled